<a href="https://colab.research.google.com/github/Nilufayeasmin299/Phishig-detection-using-both-machine-learning-and-deep-learning-techniques/blob/main/LSTM_after_phishing_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#loading dataset
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/Second paper/Dataset/My new_data')
df.head()

,Unnamed: 0,having_IP_Address,URL_Length,Shortining_Service,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Redirect,on_mouseover,RightClick,age_of_domain,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,0,-1,1,1,-1,-1,-1,-1,1,-1,1,-1,0,1,1,-1,-1,-1,1,1,-1,0
1,1,1,1,1,-1,0,1,-1,1,0,-1,-1,0,1,1,-1,0,-1,1,1,1,0
2,2,1,0,1,-1,-1,-1,-1,1,0,-1,-1,0,1,1,1,1,-1,1,0,-1,0
3,3,1,0,1,-1,-1,-1,1,-1,0,0,-1,0,1,1,-1,1,-1,1,-1,1,0
4,4,1,0,-1,-1,1,1,-1,1,0,0,-1,0,-1,1,-1,0,-1,1,1,1,1


In [3]:
df.columns

Index(['Unnamed: 0', 'having_IP_Address', 'URL_Length', 'Shortining_Service',
       'Prefix_Suffix', 'having_Sub_Domain', 'SSLfinal_State',
       'Domain_registeration_length', 'Request_URL', 'URL_of_Anchor',
       'Links_in_tags', 'SFH', 'Redirect', 'on_mouseover', 'RightClick',
       'age_of_domain', 'web_traffic', 'Page_Rank', 'Google_Index',
       'Links_pointing_to_page', 'Statistical_report', 'Result'],
      dtype='object')

In [4]:
df.shape

(11055, 22)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Unnamed: 0                   11055 non-null  int64
 1   having_IP_Address            11055 non-null  int64
 2   URL_Length                   11055 non-null  int64
 3   Shortining_Service           11055 non-null  int64
 4   Prefix_Suffix                11055 non-null  int64
 5   having_Sub_Domain            11055 non-null  int64
 6   SSLfinal_State               11055 non-null  int64
 7   Domain_registeration_length  11055 non-null  int64
 8   Request_URL                  11055 non-null  int64
 9   URL_of_Anchor                11055 non-null  int64
 10  Links_in_tags                11055 non-null  int64
 11  SFH                          11055 non-null  int64
 12  Redirect                     11055 non-null  int64
 13  on_mouseover                 11055 non-null  i

In [6]:
# shuffling the rows in the dataset so that when splitting the train and test set are equally distributed
data = df.sample(frac=1).reset_index(drop=True)
data.head()

,Unnamed: 0,having_IP_Address,URL_Length,Shortining_Service,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Request_URL,URL_of_Anchor,Links_in_tags,SFH,Redirect,on_mouseover,RightClick,age_of_domain,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,3062,1,-1,1,-1,-1,-1,1,-1,0,-1,-1,0,1,1,-1,1,-1,-1,0,1,0
1,9281,-1,-1,1,-1,-1,1,1,-1,0,0,1,0,1,1,1,1,-1,1,0,-1,1
2,166,-1,-1,-1,-1,-1,1,1,-1,0,1,-1,1,1,1,1,1,-1,1,1,1,1
3,141,1,1,1,-1,0,0,-1,1,-1,0,1,0,1,1,1,-1,-1,1,-1,1,0
4,8891,1,1,1,-1,1,1,-1,1,1,0,1,0,1,1,-1,1,1,1,0,1,1


In [7]:
# Sepratating & assigning features and target columns to X & y
y = data['Result']
X = data.drop('Result',axis=1)
X.shape, y.shape

((11055, 21), (11055,))

In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_=scaler.fit_transform(X)

In [9]:
y_=y.to_numpy()

In [10]:
X_=X_.reshape(X_.shape[0],X_.shape[1],1)

In [11]:
X_.shape

(11055, 21, 1)

In [12]:
y_.shape

(11055,)

In [13]:
import numpy
from numpy import mean
from numpy import std
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, Dropout
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
from sklearn.model_selection import KFold
from keras.layers import Flatten, Activation, LSTM
import time

In [14]:
import tensorflow as tf
import random 

In [15]:
import random
random.seed( 1 )
tf.random.set_seed(2)

In [16]:
# lists to collect the performance metrics during each fold of the n-fold cross validation
accuracy=[]
loss=[]
recall=[]
f1=[]
precision=[]
Time=[]
start = time.time()

In [17]:
epochs = 10
batch_size=100
num_folds=10
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(X_, y_):

  # Define the model architecture
  model = Sequential()
  model.add(LSTM(256, activation='relu', input_shape=(X_[0].shape)))
  model.add(Dropout(0.7))
  model.add(Dense(1, activation='sigmoid'))


  # Compile the model
  model.compile(loss='binary_crossentropy', 
                optimizer='adam',
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(X_[train], y_[train],
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)

  # Generate generalization metrics
  y_pred = model.predict_classes(X_[test])
  scores = model.evaluate(X_[test], y_[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  accuracy.append(scores[1] * 100)
  loss.append(scores[0])
  recall.append(recall_score(y_[test], y_pred))
  f1.append(f1_score(y_[test], y_pred))
  precision.append(precision_score(y_[test], y_pred))

  # Increase fold number
  fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
100/100 [==============================] - 35s 121ms/step - loss: 0.5251 - accuracy: 0.7372
Epoch 2/10
100/100 [==============================] - 12s 124ms/step - loss: 0.3607 - accuracy: 0.8448
Epoch 3/10
100/100 [==============================] - 12s 120ms/step - loss: 0.3252 - accuracy: 0.8652
Epoch 4/10
100/100 [==============================] - 12s 120ms/step - loss: 0.3058 - accuracy: 0.8728
Epoch 5/10
100/100 [==============================] - 12s 120ms/step - loss: 0.2963 - accuracy: 0.8784
Epoch 6/10
100/100 [==============================] - 12s 119ms/step - loss: 0.3019 - accuracy: 0.8799
Epoch 7/10
100/100 [==============================] - 12s 121ms/step - loss: 0.2937 - accuracy: 0.8793
Epoch 8/10
100/100 [==============================] - 12s 122ms/step - loss: 0.2650 - accuracy: 0.8925
Epoch 9/10
100/100 [==============================] - 12s 118ms/step - loss: 0.

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Score for fold 1: loss of 0.2577378451824188; accuracy of 90.05424976348877%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/10
100/100 [==============================] - 14s 123ms/step - loss: 0.5292 - accuracy: 0.7244
Epoch 2/10
100/100 [==============================] - 12s 121ms/step - loss: 0.3856 - accuracy: 0.8294
Epoch 3/10
100/100 [==============================] - 12s 123ms/step - loss: 0.3221 - accuracy: 0.8659
Epoch 4/10
100/100 [==============================] - 12s 123ms/step - loss: 0.3119 - accuracy: 0.8749
Epoch 5/10
100/100 [==============================] - 12s 120ms/step - loss: 0.2885 - accuracy: 0.8841
Epoch 6/10
100/100 [==============================] - 12s 121ms/step - loss: 0.2926 - accuracy: 0.8822
Epoch 7/10
100/100 [==============================] - 12s 120ms/step - loss: 0.2818 - accuracy: 0.8826
Epoch 8/10
100/100 [==============================] - 13s 127ms/step - loss: 0.2910 - accuracy: 0.8793
Ep

In [18]:
# obtain approximate training time
end_time = time.time()
time = (end_time - start)     #obtain time taken to train the model

In [19]:
# summarize the distribution of scores
print('Accuracy Mean: %.3f, Standard Deviation: %.3f' % (mean(accuracy), std(accuracy)))                   #mean of all the accuracies fom the n folds and standard deviation
print('Precision Mean: %.3f, Standard Deviation: %.3f' % (mean(precision), std(precision)))          #mean of all the precisions fom the n folds and standard deviation
print('Recall Mean: %.3f, Standard Deviation: %.3f' % (mean(recall), std(recall)))
print('F1 Mean: %.3f, Standard Deviation: %.3f' % (mean(f1), std(f1)))
print('Total time Mean: %.3f' % (time/10))
print('loss Mean: %.3f, Standard Deviation: %.3f' % (mean(loss), std(loss)))

Accuracy Mean: 89.751, Standard Deviation: 0.755
Precision Mean: 0.895, Standard Deviation: 0.026
Recall Mean: 0.927, Standard Deviation: 0.027
F1 Mean: 0.910, Standard Deviation: 0.006
Total time Mean: 204.749
loss Mean: 0.253, Standard Deviation: 0.019
